In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"salmanalfarisyazhar","key":"67d3b8693fe2436e1798930a8ce20de5"}'}

In [3]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/

In [4]:
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 98% 40.0M/41.0M [00:02<00:00, 28.6MB/s]
100% 41.0M/41.0M [00:02<00:00, 19.0MB/s]


In [6]:
!unzip fake-and-real-news-dataset.zip

Archive:  fake-and-real-news-dataset.zip
  inflating: Fake.csv                
  inflating: True.csv                


In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [8]:
fake_df = pd.read_csv('Fake.csv')
true_df = pd.read_csv('True.csv')

In [9]:
fake_df['label'] = 1
true_df['label'] = 0

In [10]:
df = pd.concat([fake_df, true_df], ignore_index=True)

In [11]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [12]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    words = text.split()
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(words)
    return text

In [13]:
df['text'] = df['text'].apply(preprocess_text)

In [14]:
X = df['text']
y = df['label']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [17]:
model = RandomForestClassifier()
model.fit(X_train_vec, y_train)

RandomForestClassifier()

In [19]:
y_pred = model.predict(X_test_vec)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)

print("Akurasi:", accuracy)
print("Presisi:", precision)
print("Recall:", recall)
print("F1-Score:", f1)
print("Confusion Matrix:")
print(confusion)

Akurasi: 0.9912026726057906
Presisi: 0.9925910245554614
Recall: 0.9907035706739912
F1-Score: 0.9916463994924394
Confusion Matrix:
[[4212   35]
 [  44 4689]]
